<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/agent_mistral_demo_aviation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install mistralai -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 11.9 MB/s eta 0:00:00


In [ ]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Calculator - Still relevant if an agent uses it
class CalcResult(BaseModel):
    reasoning: str
    result: str

print("Creating AI agents...")

# Agent Definitions (from original code, kept for completeness)
finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent used to answer financial related requests",
    name="finance-agent",
)

web_search_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent that can search online for any information if needed",
    name="websearch-agent",
    tools=[{"type": "web_search"}],
)

ecb_interest_rate_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Can find the current interest rate of the European central bank",
    name="ecb-interest-rate-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_european_central_bank_interest_rate",
                "description": "Retrieve the real interest rate of European central bank.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                        },
                    },
                    "required": ["date"],
                },
            },
        },
    ],
)

graph_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="graph-drawing-agent",
    description="Agent used to create graphs using the code interpreter tool.",
    instructions="Use the code interpreter tool when you have to draw a graph.",
    tools=[{"type": "code_interpreter"}],
)

calculator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

# --- Custom function for flight search (MOCK) ---
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):
    """
    MOCK: This function simulates searching for available flights.
    It returns dummy data based on specific input parameters.
    """
    print(f"\n[DEBUG] MOCK CALL: search_flights with origin='{origin}', destination='{destination}', departure_date='{departure_date}', return_date='{return_date}'")

    origin_norm = origin.upper() if origin else ""
    destination_norm = destination.upper() if destination else ""

    # Dummy data for demonstration
    if origin_norm == "YUL" and destination_norm == "JFK" and departure_date == "2025-07-01":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"},
                {"flight_number": "DL123", "airline": "Delta Airlines", "departure_time": "11:00 AM", "arrival_time": "12:45 PM", "price": "$280 CAD"}
            ]
        }
    elif origin_norm == "LAX" and destination_norm == "SFO" and departure_date == "2025-08-15":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "UA456", "airline": "United Airlines", "departure_time": "09:00 AM", "arrival_time": "10:15 AM", "price": "$120 USD"}
            ]
        }
    else:
        return {"flights_found": False, "details": "No flights found for the specified criteria."}

# --- Mock functions for refactored agents ---

def get_airport_weather(airport_code: str, date: str = None):
    """MOCK function to get weather for an airport."""
    print(f"\n[DEBUG] MOCK CALL: get_airport_weather for '{airport_code}' on '{date}'")
    if airport_code.upper() == "YUL":
        return {"status": "success", "weather": "Cloudy, 18°C, light rain expected.", "forecast_date": date or "2025-06-21"}
    return {"status": "success", "weather": "Clear skies, 25°C.", "forecast_date": date or "2025-06-21"}

def check_aircraft_maintenance_status(aircraft_tail_number: str):
    """MOCK function to check aircraft maintenance status."""
    print(f"\n[DEBUG] MOCK CALL: check_aircraft_maintenance_status for '{aircraft_tail_number}'")
    if aircraft_tail_number.upper() == "C-FROY":
        return {"status": "ok", "next_maintenance": "2025-07-15", "details": "Routine check passed."}
    return {"status": "ok", "next_maintenance": "N/A", "details": "No issues found."}

def report_maintenance_issue(aircraft_tail_number: str, issue_description: str, severity: str):
    """MOCK function to report a maintenance issue."""
    print(f"\n[DEBUG] MOCK CALL: report_maintenance_issue for '{aircraft_tail_number}' - '{issue_description}' (Severity: {severity})")
    return {"status": "issue_reported", "issue_id": "MNT-12345", "aircraft": aircraft_tail_number}

def get_crew_availability(crew_type: str, date: str):
    """MOCK function to get crew availability."""
    print(f"\n[DEBUG] MOCK CALL: get_crew_availability for '{crew_type}' on '{date}'")
    if crew_type.lower() == "pilot" and date == "2025-07-01":
        return {"status": "success", "available_crew_ids": ["PILOT-001", "PILOT-002"]}
    return {"status": "success", "available_crew_ids": []}

def assign_crew_to_flight(flight_number: str, crew_ids: list):
    """MOCK function to assign crew to a flight."""
    print(f"\n[DEBUG] MOCK CALL: assign_crew_to_flight '{flight_number}' with crew: {crew_ids}")
    return {"status": "success", "message": f"Crew {crew_ids} assigned to flight {flight_number}."}

def get_booking_status(booking_reference: str):
    """MOCK function to get booking status."""
    print(f"\n[DEBUG] MOCK CALL: get_booking_status for '{booking_reference}'")
    if booking_reference.upper() == "ABCDEF":
        return {"status": "confirmed", "passenger_name": "John Doe", "flight_number": "AC700"}
    return {"status": "not_found", "message": "Booking reference not found."}

def process_special_request(booking_reference: str, request_details: str):
    """MOCK function to process a special request."""
    print(f"\n[DEBUG] MOCK CALL: process_special_request for '{booking_reference}' - '{request_details}'")
    return {"status": "request_logged", "details": request_details}

def get_flight_status(flight_number: str, date: str = None):
    """MOCK function to get flight status."""
    print(f"\n[DEBUG] MOCK CALL: get_flight_status for '{flight_number}' on '{date}'")
    if flight_number.upper() == "AC700" and (date == "2025-07-01" or date is None):
        return {"status": "delayed", "reason": "Weather", "new_eta": "12:00 PM"}
    return {"status": "on_time"}

def issue_delay_notification(flight_number: str, delay_reason: str, new_estimated_time: str):
    """MOCK function to issue a delay notification."""
    print(f"\n[DEBUG] MOCK CALL: issue_delay_notification for '{flight_number}' - Reason: '{delay_reason}', New ETA: '{new_estimated_time}'")
    return {"status": "notification_sent", "flight": flight_number, "details": f"Delayed due to {delay_reason}, new ETA: {new_estimated_time}"}


# --- Refactored Flight Planning Related Agents ---

print("Refactoring existing agents for flight planning context...")

# 1. Flight Planning Agent (Original)
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with flight planning requests, searching for flights and providing details.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Optional return date for round trips.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., 'YUL')."},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., 'JFK')."},
                        "departure_date": {"type": "string", "description": "Departure date in YYYY-MM-DD format."},
                        "return_date": {"type": "string", "description": "Return date in YYYY-MM-DD format (optional)."}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        {"type": "web_search"} # For general web searches related to flight planning
    ]
)
print(f"Flight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")
tool_names_list_fp = []
for tool in flight_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list_fp.append(tool.function.name)
    else:
        tool_names_list_fp.append(tool.type)
print(f"Tools available to flight_planning_agent: {tool_names_list_fp}")


# 2. Weather Agent
weather_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for retrieving real-time weather information relevant to flight planning, including forecasts, adverse conditions, and airport weather.",
    name="weather-planning-agent",
    tools=[
        {"type": "web_search"},
        {
            "type": "function",
            "function": {
                "name": "get_airport_weather",
                "description": "Retrieve current weather conditions and short-term forecast for a specific airport.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "airport_code": {"type": "string", "description": "IATA code of the airport (e.g., 'YUL')."},
                        "date": {"type": "string", "description": "Date for the forecast in YYYY-MM-DD format (optional)."}
                    },
                    "required": ["airport_code"]
                }
            }
        }
    ]
)
print(f"Weather Planning Agent '{weather_agent.name}' created with ID: {weather_agent.id}")
tool_names_list_weather = []
for tool in weather_agent.tools:
    if tool.type == 'function':
        tool_names_list_weather.append(tool.function.name)
    else:
        tool_names_list_weather.append(tool.type)
print(f"Tools available to weather_planning_agent: {tool_names_list_weather}")


# 3. Maintenance Agent
maintenance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking aircraft maintenance schedules, status, and reporting potential issues that could affect flight readiness.",
    name="maintenance-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_aircraft_maintenance_status",
                "description": "Check the maintenance status and next scheduled maintenance for a given aircraft tail number.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_tail_number": {"type": "string", "description": "The tail number of the aircraft (e.g., 'C-FROY')."}
                    },
                    "required": ["aircraft_tail_number"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "report_maintenance_issue",
                "description": "Report a new maintenance issue for an aircraft, including description and severity.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aircraft_tail_number": {"type": "string", "description": "The tail number of the aircraft."},
                        "issue_description": {"type": "string", "description": "A description of the maintenance issue."},
                        "severity": {"type": "string", "description": "The severity of the issue (e.g., 'critical', 'minor')."}
                    },
                    "required": ["aircraft_tail_number", "issue_description", "severity"]
                }
            }
        }
    ]
)
print(f"Maintenance Planning Agent '{maintenance_agent.name}' created with ID: {maintenance_agent.id}")
tool_names_list_maintenance = []
for tool in maintenance_agent.tools:
    if tool.type == 'function':
        tool_names_list_maintenance.append(tool.function.name)
    else:
        tool_names_list_maintenance.append(tool.type)
print(f"Tools available to maintenance_planning_agent: {tool_names_list_maintenance}")


# 4. Crew Planning Agent
crew_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing and coordinating flight crew assignments, checking crew availability, and managing roster changes.",
    name="crew-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_crew_availability",
                "description": "Retrieve the availability of flight crew members for a specific date or flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "crew_type": {"type": "string", "description": "Type of crew (e.g., 'pilot', 'flight attendant')."},
                        "date": {"type": "string", "description": "Date for availability check in YYYY-MM-DD format."}
                    },
                    "required": ["crew_type", "date"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assign_crew_to_flight",
                "description": "Assign specific crew members to a flight.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_number": {"type": "string", "description": "The flight number."},
                        "crew_ids": {"type": "array", "items": {"type": "string"}, "description": "List of crew member IDs to assign."}
                    },
                    "required": ["flight_number", "crew_ids"]
                }
            }
        }
    ]
)
print(f"Crew Planning Agent '{crew_planning_agent.name}' created with ID: {crew_planning_agent.id}")
tool_names_list_crew = []
for tool in crew_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list_crew.append(tool.function.name)
    else:
        tool_names_list_crew.append(tool.type)
print(f"Tools available to crew_planning_agent: {tool_names_list_crew}")


# 5. Passenger Coordination Agent
passenger_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with passenger-related inquiries, including booking status, special requests, and re-routing due to delays.",
    name="passenger-coordination-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_booking_status",
                "description": "Retrieve the current status of a passenger booking using a booking reference.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "booking_reference": {"type": "string", "description": "The passenger's booking reference number."}
                    },
                    "required": ["booking_reference"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "process_special_request",
                "description": "Process a special request from a passenger (e.g., wheelchair assistance, special meal).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "booking_reference": {"type": "string", "description": "The passenger's booking reference number."},
                        "request_details": {"type": "string", "description": "Description of the special request."}
                    },
                    "required": ["booking_reference", "request_details"]
                }
            }
        }
    ]
)
print(f"Passenger Coordination Agent '{passenger_coordination_agent.name}' created with ID: {passenger_coordination_agent.id}")
tool_names_list_passenger = []
for tool in passenger_coordination_agent.tools:
    if tool.type == 'function':
        tool_names_list_passenger.append(tool.function.name)
    else:
        tool_names_list_passenger.append(tool.type)
print(f"Tools available to passenger_coordination_agent: {tool_names_list_passenger}")


# 6. Operations Agent
operations_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for overseeing overall flight operations, monitoring flight statuses, managing delays, and coordinating with various departments.",
    name="operations-agent",
    tools=[
        {"type": "web_search"},
        {
            "type": "function",
            "function": {
                "name": "get_flight_status",
                "description": "Retrieve the real-time status of a specific flight (e.g., on-time, delayed, cancelled).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_number": {"type": "string", "description": "The flight number (e.g., 'AC700')."},
                        "date": {"type": "string", "description": "The date of the flight in YYYY-MM-DD format (optional, defaults to today)."}
                    },
                    "required": ["flight_number"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "issue_delay_notification",
                "description": "Issue a notification for a flight delay, specifying the reason and new estimated time.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "flight_number": {"type": "string", "description": "The flight number."},
                        "delay_reason": {"type": "string", "description": "Reason for the delay."},
                        "new_estimated_time": {"type": "string", "description": "New estimated departure or arrival time."}
                    },
                    "required": ["flight_number", "delay_reason", "new_estimated_time"]
                }
            }
        }
    ]
)
print(f"Operations Agent '{operations_agent.name}' created with ID: {operations_agent.id}")
tool_names_list_ops = []
for tool in operations_agent.tools:
    if tool.type == 'function':
        tool_names_list_ops.append(tool.function.name)
    else:
        tool_names_list_ops.append(tool.type)
print(f"Tools available to operations_agent: {tool_names_list_ops}")

print("\nAll new flight-related agents have been defined.")

# --- Master Tool Executor Mapping ---
# This dictionary will map tool names to their corresponding local mock functions
tool_executor = {
    "search_flights": search_flights,
    "get_airport_weather": get_airport_weather,
    "check_aircraft_maintenance_status": check_aircraft_maintenance_status,
    "report_maintenance_issue": report_maintenance_issue,
    "get_crew_availability": get_crew_availability,
    "assign_crew_to_flight": assign_crew_to_flight,
    "get_booking_status": get_booking_status,
    "process_special_request": process_special_request,
    "get_flight_status": get_flight_status,
    "issue_delay_notification": issue_delay_notification,
    # Add other mock functions here as needed for other agents
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: Information retrieved."
}

# --- Function to standardize tools for client.chat.complete ---
def get_api_call_tools_list(agent_tools):
    """
    Converts agent.tools objects into the specific dictionary format
    expected by client.chat.complete. Handles 'web_search' workaround.
    """
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            # WORKAROUND: Force 'web_search' to have 'type: "function" and a dummy 'function' definition
            # to satisfy the API validator. The actual execution will be handled by tool_executor.
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool", # A dummy function name
                    "description": "Accesses the internet to find information.",
                    "parameters": { # Parameters are usually dynamic for web_search, so empty object
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for ALL Flight-Related Agents (via chat completions) ---
# This simulates a multi-turn conversation where your code acts as the tool executor.

print("\n--- Executing Test Cases for Flight-Related Agents (via chat completions) ---")

test_cases = [
    {
        "agent": flight_planning_agent,
        "name": "Flight Planning Agent",
        "query": "Find me a flight from Montreal to New York on July 1st, 2025.",
        "expected_tool_call": "search_flights"
    },
    {
        "agent": weather_agent,
        "name": "Weather Agent",
        "query": "What's the weather like at YUL for tomorrow?",
        "expected_tool_call": "get_airport_weather"
    },
    {
        "agent": maintenance_agent,
        "name": "Maintenance Agent",
        "query": "Check the maintenance status for aircraft C-FROY.",
        "expected_tool_call": "check_aircraft_maintenance_status"
    },
    {
        "agent": crew_planning_agent,
        "name": "Crew Planning Agent",
        "query": "Are there any pilots available on July 1st, 2025?",
        "expected_tool_call": "get_crew_availability"
    },
    {
        "agent": passenger_coordination_agent,
        "name": "Passenger Coordination Agent",
        "query": "What's the booking status for reference ABCDEF?",
        "expected_tool_call": "get_booking_status"
    },
    {
        "agent": operations_agent,
        "name": "Operations Agent",
        "query": "What's the current status of flight AC700 for July 1st, 2025?",
        "expected_tool_call": "get_flight_status"
    },
     {
        "agent": operations_agent,
        "name": "Operations Agent (Web Search Example)",
        "query": "What's the latest news on air traffic control strikes in France?",
        "expected_tool_call": "internal_web_search_tool" # This will trigger the mock web search
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"]

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list,
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")

Mounted at /content/gdrive
Creating AI agents...
Refactoring existing agents for flight planning context...
Flight Planning Agent 'flight-planning-agent' created with ID: ag_06856940594577378000007fb4c23ff9
Tools available to flight_planning_agent: ['search_flights', 'web_search']
Weather Planning Agent 'weather-planning-agent' created with ID: ag_068569405cd5789b8000fe9db085495e
Tools available to weather_planning_agent: ['web_search', 'get_airport_weather']
Maintenance Planning Agent 'maintenance-planning-agent' created with ID: ag_06856940604777ac8000c24bf242e9b4
Tools available to maintenance_planning_agent: ['check_aircraft_maintenance_status', 'report_maintenance_issue']
Crew Planning Agent 'crew-planning-agent' created with ID: ag_0685694063c37e698000748f0d7fc211
Tools available to crew_planning_agent: ['get_crew_availability', 'assign_crew_to_flight']
Passenger Coordination Agent 'passenger-coordination-agent' created with ID: ag_068569406720796a8000f4e6dd6cd3e7
Tools available